In [4]:
from pathlib import Path
import numpy as np
import pandas as pd
import os 
import geemap
import ee
from typing import Dict, List, Tuple
from tqdm import tqdm
from utils import extractROI


ee.Authenticate()
ee.Initialize(project="virtual-rarity-426212-p6")
Map = geemap.Map()


LAI_BAND = 'Lai_500m'
FOREST_BAND = 'Percent_Tree_Cover'
GREEN_BAND = 'NDVI'
ZIPPED_FHAPEFILES_DIR = "./zipped_files"
START_DATE = ee.Date('2000-02-18')
END_DATE = ee.Date('2023-01-01')
ASSET_FODLER = "projects/virtual-rarity-426212-p6/assets/shapefiles"

# Each zipfile is FeatureCollection type asset





class Vegetation():
    
    def __init__(self, start_date: str, end_date: str, land_cover: str = 'MODIS/061/MOD15A2H'):
        
        self.start_date = start_date
        self.end_date = end_date 
        self.landcover_dataset = ee.ImageCollection(land_cover).\
                    filterDate(start_date, end_date).select(LAI_BAND)
                    
        self.forest_fraction = ee.ImageCollection("MODIS/006/MOD44B").\
                    filterDate('2000-03-05', '2020-03-05').select(FOREST_BAND)
                    
        self.green_fraction = ee.ImageCollection("MODIS/061/MOD13Q1").\
                    filterDate(start_date, end_date).select(GREEN_BAND)
        
        self.roi_lst, self.basin_names = extractROI(ASSET_FODLER)
        self.months  = ee.List.sequence(1, 12)
     
        
    
    def _compute_monthly_lai_stats(self, month: str)->ee.ImageCollection:
        
        monthlyLai = self.landcover_dataset.filter(ee.Filter.calendarRange(month, month, 'month'))
        
        monthly_mean_lai = monthlyLai.mean().rename('lai_mean')
        monthly_max_lai = monthlyLai.max().rename('lai_max') # max of a month across the years 
        monthly_min_lai = monthlyLai.min().rename('lai_min') # min of a month across the years 
        
        monthly_stats_lai = monthly_mean_lai.addBands([monthly_max_lai, monthly_min_lai])
        
        return monthly_stats_lai.set('month', month)
        
        
    def _computeOneLaiStats(self, roi: ee.Geometry)->Tuple[float, float]:
        
        monthly_lai_stats = self.landcover_dataset.\
                            fromImages(self.months.map(lambda month: self._compute_monthly_lai_stats(month)))
        
        
        clipped_lai_images =  monthly_lai_stats.map(lambda img: img.clip(roi))
        lai_mean_images = clipped_lai_images.select('lai_mean')
        
    
        max_lai_mean = lai_mean_images.reduce(ee.Reducer.max()).reduceRegion(
                            reducer=ee.Reducer.mean(),
                            geometry=roi,
                            scale=500,  
                            maxPixels=1e9
                            )
        
        min_lai_mean = lai_mean_images.reduce(ee.Reducer.min()).reduceRegion(
                            reducer=ee.Reducer.mean(),
                            geometry=roi,
                            scale=500,  
                            maxPixels=1e9
                            )
        
        num_max_lai_mean = max_lai_mean.getInfo()['lai_mean_max']
        num_min_lai_mean = min_lai_mean.getInfo()['lai_mean_min']
        mean_diff = num_max_lai_mean - num_min_lai_mean
        
        return round(num_max_lai_mean/4, 7), round(mean_diff/4,7)  # 31/8-day composite
    
    
    
    def _computeForestFractionStats(self, roi: ee.Geometry)->float:
        
        clipped_tree_cover = self.forest_fraction.map(lambda img: img.clip(roi))
        
        mean_tree_cover = clipped_tree_cover.reduce(ee.Reducer.mean()).reduceRegion(
                            reducer = ee.Reducer.mean(),
                            geometry=roi,
                            scale=250,
                            maxPixels=1e9
                          )
        
        return round(mean_tree_cover.getInfo()['Percent_Tree_Cover_mean']/100, 7)
    
    # aggregate green fractions for a month across the years 
    def _computeOneGreenFraction(self, month: ee.Number)->ee.Image:
        
        monthly_ndvi = self.green_fraction.filter(ee.Filter(month, month, 'month'))
        mean_monthly_ndvi = monthly_ndvi.mean().rename['mean_ndvi']
        return mean_monthly_ndvi.set('month', month)
    
    
    def _computeGreenFractionStats(self, roi: ee.Geometry)->float:
        
        mean_monthly_ndvi = self.green_fraction.\
                            fromImages(self.months.map(lambda month: self._computeOneGreenFraction(month)))
        
        clipped_imgs = mean_monthly_ndvi.map(lambda img: img.clip(roi)).select('mean_ndvi')
        # Reduce the collection to find the max/min value as one image
        max_gree_fraction = clipped_imgs.reduce(ee.Reducer.max())
        min_gree_fraction = clipped_imgs.reduce(ee.Reducer.min())
        
        gvf_formula = '((ndvi - ndvi_min) / (ndvi_max - ndvi_min))'

        # Apply the formula to calculate GVF of each image (converted image collection)
        gvf = clipped_imgs.map(lambda img: img.expression(
            gvf_formula,
            {
                'ndvi': img.select('mean_ndvi'),
                'ndvi_min': min_gree_fraction.select('mean_ndvi_min'),
                'ndvi_max': max_gree_fraction.select('mean_ndvi_max')
            }).rename('GVF')
        )
        
        # Reduce the mean_image to find the mean value within the ROI
        gvf_mean = gvf.reduceRegion(
                    reducer = ee.Reducer.mean(),
                    geometry = roi, 
                    scale = 250,
                    maxPixels=1e9
                    )
        
        return max_gree_fraction.getInfo(), gvf.getInfo()
        
        
    def computeAllStats(self):
        
        table = {"lai_max": [], "lai_diff": [], "forest_frac": []}
        
        for idx, roi in tqdm(enumerate(self.roi_lst)):
            
            if idx==5:
                break
            lai_max, lai_diff = self._computeOneLaiStats(roi)
            forest_frac = self._computeForestFractionStats(roi)
            table['lai_max'].append(lai_max)
            table['lai_diff'].append(lai_diff)
            table['forest_frac'].append(forest_frac)
            
        dataframe = pd.DataFrame(table, index=self.basin_names[:5])
        return dataframe
    
    
    
    

vegi = Vegetation(START_DATE, END_DATE)
result = vegi._computeGreenFractionStats(vegi.roi_lst[0])
print(result)

TypeError: Filter.__init__() takes from 1 to 2 positional arguments but 4 were given